In [ ]:
import numpy as np
import pandas as pd

from utils.drl_train import (
    generate_sliding_windows,
    train_agents,
    select_best_agent,
    backtest_all_agents
)
from utils.portfolio_env_old import PortfolioEnv
from utils.drl_agent import DRLAgent

In [2]:
# --- Load your data ---
returns_df = pd.read_parquet("../data/returns.parquet")
prices_df = pd.read_parquet("../data/prices.parquet")
vol_df = pd.read_parquet("../data/vola.parquet")

In [ ]:
# --- Pick the first sliding window: 2006–2012 (train), 2011 (val), 2012 (test) ---
window = list(generate_sliding_windows(2006, 2021))[0]
print("Window:", window)

Window: {'train': ('2006-01-01', '2011-01-01'), 'val': ('2011-01-01', '2012-01-01'), 'test': ('2012-01-01', '2013-01-01')}


In [ ]:
# --- Train 5 agents ---
agents = train_agents(DRLAgent, PortfolioEnv, returns_df, prices_df, vol_df, window, n_seeds=5)

# --- Select best agent based on Sharpe in validation year ---
best_agent = select_best_agent(agents, PortfolioEnv, returns_df, prices_df, vol_df, window)

# --- Backtest all 5 agents on the test year (2012) ---
all_metrics = backtest_all_agents(agents, PortfolioEnv, returns_df, prices_df, vol_df, window)

# --- Summarize Results ---
def summarize_metrics(metrics_list):
    summary = {}
    keys = metrics_list[0].keys()
    for key in keys:
        values = [m[key] for m in metrics_list]
        if isinstance(values[0], (int, float)):
            summary[key] = {
                "mean": np.mean(values),
                "std": np.std(values)
            }
    return summary

summary = summarize_metrics(all_metrics)

print("\n📊 Aggregated Evaluation Metrics (Mean ± Std over 5 agents):")
for k, v in summary.items():
    print(f"{k}: {v['mean']:.4f} ± {v['std']:.4f}")

ValueError: NaN values in observation at step 60